# Football Analytics & Machine Learning

Proyecto de análisis de fútbol aplicando **clustering**, **clasificación** y **recomendación** de jugadores a partir de estadísticas de rendimiento (FBRef) e información de mercado (Transfermarkt).

> Nota: esta notebook es una versión “portfolio-ready” del trabajo original. Se prioriza claridad, reproducibilidad y resultados.


## Cómo ejecutar

1. Instalar dependencias (ver `requirements.txt` en el repositorio).
2. Abrir esta notebook y ejecutar las celdas en orden.
3. Asegurarse de contar con los archivos de datos en la carpeta indicada en la sección **Datos**.

## Datos

- **FBRef**: estadísticas de rendimiento por jugador/temporada.
- **Transfermarkt**: información de valor de mercado por jugador.

Los datasets se integran mediante identificadores/nombres normalizados (según disponibilidad), y se aplican filtros mínimos (por ejemplo, minutos jugados) para evitar observaciones poco representativas.


## Francisco Borda Rojas - Julio 2024

In [ ]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt


from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.preprocessing import MinMaxScaler 
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import train_test_split, KFold
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from formulaic import Formula

import tensorflow as tf
import keras
from tf_regressor import train_test_split_scale_center

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

### Preprocesamiento

##

Cargamos el archivo "FBref2020-21.csv" en un DataFrame

In [ ]:
futbol = pd.read_csv("FBRef2020-21.csv")
futbol.head()

##

Eliminamos los jugadores que jugaron menos de 500 minutos en la temporada

In [ ]:
futbol = futbol[futbol['Min'] >= 500]

##

Ahora, eliminamos las columnas con más de 100 datos faltantes y eliminamos las filas con datos faltantes

In [ ]:
# Eliminar columnas con más de 100 datos faltantes
futbol = futbol.loc[:, futbol.isnull().sum() <= 100]

# Eliminar ahora las filas con datos faltantes
futbol = futbol.dropna()

futbol.head()

##

Reseteamos los índices

In [ ]:
futbol.reset_index(drop=True, inplace= True)
futbol.head()

##

Definimos el DataFrame data_num que solo contenga las variables númericas, a partir de la columna Ast/90, inclusive. A pedido del docente, para clustering y clasificación no vamos a utilizar las variables categóricas ni edad ni minutos jugados.


In [ ]:
# Armamos DataFrame data_num

# Seleccionamos el indice de la columna Ast/90
indice_Ast90 = futbol.columns.get_loc('Ast/90')

# Seleccionamos todas las columnas a partir de Ast/90 y luego nos quedamos solo con las que tienen datos numéricos
data_num = futbol.iloc[:, indice_Ast90:].select_dtypes(include=[float, int])
data_num

### Clustering

##

### Objetivo
Seleccionar dos variables cualesquiera de los datos y realizar un gráfico de dispersión de una variable en función de la otra para el total de las observaciones

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
(
    so.Plot(data=data_num, x="npG+A/90", y="Shots/90")
    .add(so.Dot())
    .layout(size=(10,6))
)

En este grafico observamos sobre el eje X los goles + asistencias de los diferentes jugadores, y en el eje Y, los tiros al arco de los diferentes jugadores. Estas dos estadisticas están basadas en un promedio de cada 90 minutos. No encontramos facilmente grupos distintos.

### Objetivo
Escalar los datos y realizar un análisis de componentes principales, quedándose solo con las dos primeras componentes. Realizar un gráfico como el del punto ant

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
# Escalamos los datos
data = StandardScaler().fit_transform(data_num)

# Aplicamos PCA
pca = PCA(n_components=2)

componentes = pca.fit_transform(data)
pca_data_num = pd.DataFrame(data=componentes, columns=["PC1", "PC2"])

pca_data_num.head()

Ya realizado el análisis de componentes principales vamos a realizar el grafico.

In [ ]:
(
    so.Plot(data=pca_data_num, x="PC1", y="PC2")
    .add(so.Dot())
    .label(title="PCA - Dos primeras componentes")
    .layout(size=(10,8))
)

Observamos que podemos identificar principalmente dos clusters, uno muy grande que ocupa casi todo el grafico y uno muy chico que aparece en la parte baja del grafico. Si miramos más en detalle, podemos dividir el cluster grande en 2/3 clusters. Es decir, al final podemos visualizar hasta 4 clusters. A continuación vamos a crear estos clusters manualmente.

In [ ]:
# Formo manualmente los clusters

clusters = [
    pca_data_num["PC2"] < -8,
    (pca_data_num["PC2"] >= -8) & (pca_data_num["PC1"] < -3),
    (pca_data_num["PC1"] >= -3) & (pca_data_num["PC1"] <= 3),
    pca_data_num["PC1"] > 3
]

opc = [0, 1, 2, 3]

# Asignar clusters basados en las condiciones
pca_data_num["Cluster"] = np.select(clusters, opc, default=-1)

In [ ]:
(
    so.Plot(data=pca_data_num, x="PC1", y="PC2", color = "Cluster")
    .add(so.Dot())
    .label(title="PCA - Dos primeras componentes")
    .layout(size=(10,8))
)

Vemos más claramente los clusters que mencionamos anteriormente. Ahora vamos a ver si podemos darnos cuenta a que características responden cada cluster.

In [ ]:
data_num['Cluster'] = pca_data_num['Cluster']
cluster_analisis = data_num.groupby('Cluster').mean()

In [ ]:
display(
    so.Plot(data=cluster_analisis, x="npxG+xA/90", y="Interceptions/90", color='Cluster')
    .add(so.Dot())
    .label(title="Clusters basados en expectativa de goles y asistencias (cada 90 mins), y pases progresivos (cada 90 mins)", x="Goles y asistencias esperados c/ 90 mins", y="Intercepciones c/ 90 mins")
    .layout(size=(10, 8))
)

display(
    so.Plot(data=cluster_analisis, x="Clearances/90", y="SuccDrib/90", color='Cluster')
    .add(so.Dot())
    .label(title="Clusters basados en despejes (cada 90 mins), y gambetas exitosas (cada 90 mins)", x="Despejes c/ 90 mins", y="Gambetas exitosas c/ 90 mins")
    .layout(size=(10, 8))
)

### Grafico 1: 
Lo más seguro es que el cluster 0 pertenezca a los arqueros, ya que no cuenta ni con un alto numero de goles y asistencias esperadas cada 90 minutos, ni con un alto numero de intercepciones cada 90 minutos. Esto quiere decir que no defiende ni ataca, por lo que nos quedan los arqueros. El cluster 3 es bastante facil de reconocer tambien ya que al ser el cluster que más alto valor tiene en el eje x podemos asegurar que muy probablemente este sea el de los delanteros. Por último nos quedan los clusters 1 y 2. Creemos que el 1 es el de los defensores ya que cuenta con un mayor valor en el eje de intercepciones y menor en el eje de goles y asistencias. El último cluster (2) es el de muy probablemente los mediocampistas (están bastante arriba en el eje y, y casi en el medio del eje x).
### Grafico 2:
Podemos confirmar que el cluster 0 pertenece a los arqueros y el cluster 3 a los delanteros. También podemos decir que el cluster 2 pertenece a los mediocampistas ya que están bastante equilibrados en ambos graficos. Por último, el cluster 1 sería el de los defensores ya que siempre tienen un valor alto en el eje más defensivo.
### Conclusión:
El cluster 0 tiene caracteristicas muy pobres en estos graficos porque ahi se encuentran los arqueros.
El cluster 1 tiene caracteristicas muy defensivas asique debe pertenecer a los defensores.
El cluster 2 es el de características más equilibradas, por lo que debe pertenecer a los medios.
Para concluir, el cluster 3 tiene características muy ofensivas (es decir pertenece a los delanteros).

### Objetivo
Para la cantidad de clusters observados en el ítem anterior, realizar un agrupamiento por k-medias, y colorear los puntos según las etiquetas obtenidas. ¿Coinci

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
# Definimos la cantidad de clusters que buscamos
kmeans = KMeans(n_clusters=4)

# Ajustamos el modelo.
kmeans.fit(pca_data_num[["PC1","PC2"]])
labels = kmeans.labels_

# Añadimos los labels al dataframe
pca_data_num["Cluster"] = labels

(
    so.Plot(data=pca_data_num, x="PC1", y="PC2", color="Cluster")
    .add(so.Dot())
    .label(title="Gráfico de Dispersión de las dos Primeras Componentes Principales con Clusters")
    .layout(size=(10,8))
)

Luego de realizar un agrupamiento por k-medias, vemos que el resultado no es el que habiamos analizado anteriormente. Vemos que hay cuatro clusters pero no son los que vimos. Esto se debe a que el metodo de agrupamiento por k-medias consiste en encontrar centros para "k" clusters, y por lo tanto era esperable que los datos sean divididos de igual manera.

### Objetivo
Repetir el agrupamiento utilizando DBSCAN. ¿Cómo eligirían en este caso un valor de ε apropiado

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


Antes de realizar el agrupamiento por DBSCAN debemos encontrar el mejor parametro para eps (distancia entre puntos). Para esto vamos a realizar el siguiente metodo:

In [ ]:
# Ponemos "n_neighbors=2" ya que cada punto se incluye a si mismo en la busqueda del más cercano.
cercanos = NearestNeighbors(n_neighbors=2)
cercanos = cercanos.fit(pca_data_num[["PC1","PC2"]])

distances, indices = cercanos.kneighbors(pca_data_num[["PC1","PC2"]])

# Ignoramos la primer distancia que es la que tiene el punto con él mismo.
distances = distances[:,1]

# Ordenamos en orden ascendente.
distances = np.sort(distances, axis=0)

(
    so.Plot(x = np.arange(len(distances)), y = distances)
    .add(so.Line())
)

Sabemos que el mejor valor es cuando se lleva a cabo la curva fuerte, es decir en aproximadamente 0.7. Procedemos a realizar el codigo.

In [ ]:
# Elegimos un valor para minimo de puntos bastante alto. Luego vamos a elegir uno más chico y comparamos.
dbscan = DBSCAN(eps=0.7, min_samples=25)
dbscan.fit(pca_data_num[["PC1","PC2"]])
labels = dbscan.labels_

pca_data_num["Cluster"] = labels

(
    so.Plot(data=pca_data_num, x="PC1", y="PC2", color="Cluster")
    .add(so.Dot())
    .label(title="Gráfico de Dispersión de las dos Primeras Componentes Principales con Clusters")
    .layout(size=(10,8))
)

In [ ]:
print(f"El numero de clusters obtenidos con min_samples= 25 es: {pca_data_num['Cluster'].max()+1}")

In [ ]:
dbscan = DBSCAN(eps=0.7, min_samples=5)
dbscan.fit(pca_data_num[["PC1","PC2"]])
labels = dbscan.labels_

pca_data_num["Cluster"] = labels

(
    so.Plot(data=pca_data_num, x="PC1", y="PC2", color="Cluster")
    .add(so.Dot())
    .label(title="Gráfico de dispersión de las dos primeras Componentes Principales con clusters")
    .layout(size=(10,8))
)

In [ ]:
print(f"El numero de clusters obtenidos con min_samples= 5 es: {pca_data_num['Cluster'].max()+1}")

Sacamos algunas conclusiones. En primer lugar, vemos que en ningun caso nos quedan los cuatro clusters que nos imaginabamos al principio. Luego, vemos que al usar un valor alto de puntos minimos, tenemos menor cantidad de clusters (6), y si usamos un valor más bajo tenemos una mayor cantidad de clusters (8), pero la diferencia es notablemente baja. Esto es algo esperado ya que al ver el grafico observamos que los puntos están sumamente cerca por lo que la cantidad minima de puntos no es algo que afecte demasiado.

### Objetivo
Aplicar DBSCAN para realizar agrupamiento utilizando como datos todas las variables originales en vez de solo las dos componentes principales, modificando los v

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


Lo que entendí de este ejercicio es que tengo que buscar el hiperparametro epsilon pero ahora ajustando con todos los componentes principales y no solo con los 2 primeros.

In [ ]:
data2 = StandardScaler().fit_transform(data_num)

# Aplicamos PCA
pca = PCA()

componentes = pca.fit_transform(data2)
pca_data_num2 = pd.DataFrame(data=componentes)

for i, col in enumerate(pca_data_num2.columns):
    pca_data_num2.rename(columns={col: f"PC{i}"}, inplace=True)

pca_data_num2.head()

In [ ]:
cercanos = NearestNeighbors(n_neighbors=2)

cercanos = cercanos.fit(pca_data_num2[pca_data_num2.columns])

distances, indices = cercanos.kneighbors(pca_data_num2[pca_data_num2.columns])

distances = distances[:,1]

distances = np.sort(distances, axis=0)

(
    so.Plot(x = np.arange(len(distances)), y = distances)
    .add(so.Line())
)

Usamos hiperparametro = 3 y luego hiperparametro = 9.

In [ ]:
dbscan = DBSCAN(eps=3, min_samples=5)
dbscan.fit(pca_data_num2[pca_data_num2.columns])
labels = dbscan.labels_

pca_data_num2["Cluster"] = labels

(
    so.Plot(data=pca_data_num2, x="PC1", y="PC2", color="Cluster")
    .add(so.Dot())
    .label(title="Gráfico de dispersión con todas las Componentes Principales con clusters")
    .layout(size=(10,8))
)

In [ ]:
print(f"El numero de clusters obtenidos con min_samples= 5 es: {pca_data_num2['Cluster'].max()+1}")

In [ ]:
dbscan = DBSCAN(eps=9, min_samples=15)
dbscan.fit(pca_data_num2[pca_data_num2.columns])
labels = dbscan.labels_

pca_data_num2["Cluster"] = labels

(
    so.Plot(data=pca_data_num2, x="PC1", y="PC2", color="Cluster")
    .add(so.Dot())
    .label(title="Gráfico de dispersión con todas las Componentes Principales con clusters")
    .layout(size=(10,8))
)

In [ ]:
print(f"El numero de clusters obtenidos con min_samples= 5 es: {pca_data_num2['Cluster'].max()+1}")

### Objetivo
anterior.

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


### Clasificación

##

Ahora queremos poder predecir la posición en la que juega cada jugador según sus datos estadísticos utilizando KNN. En la columna Pos encontramos la posición de los jugadores. Para la mayoría de los jugadores se indica una única posición pero algunos jugadores tienen dos posiciones. Para simplificar el análisis vamos a considerar una única posición por jugador.

### Objetivo
Definir la variable Pos_filt que es la columna Pos, pero donde los jugadores deben tener una sola posición (pueden quedarse sólo con la primera posición de cada

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
# Definimos la variable Pos_filt que son los datos de la columna Pos pero cada jugador tiene solo una posición.
futbol['Pos_filt'] = futbol['Pos'].str.split(',').str[0] #dividimos la cadena de posiciones que estan separadas (de haber 2) por una coma y nos quedamos solo con la primera de las Pos.
futbol[['Player','Pos','Pos_filt']]

### Objetivo
Dividir el DataFrame data_num y la Series Pos_filt utilizando un 80% para entrenamiento y un 20% para testeo.

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
X = data_num
y = futbol['Pos_filt']
X = MinMaxScaler().set_output(transform='pandas').fit_transform(X)
#Separo el DataFrame data_num y la Serie Pos_filt en entrenamineto (80%) y testeo(20%)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state= 42)

### Objetivo
Aplicar un esquema de validación en el conjunto de entrenamiento para seleccionar el valor óptimo de K. (Esto puede demorar mucho si prueban muchos valores de K

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
cv_k_scores = []
for K in range(1,21): # probamos con K del 1 al 20
    neighbor = KNeighborsClassifier(n_neighbors=K) 
    scores = cross_val_score(neighbor, X_train, y_train) #Hacemos validación cruzada con cada K en los conjuntos de entrenamiento.
    cv_k_scores.append(scores.mean()) # Hace un promedio de los valores de score obtenidos en las particiones del cross validation.
    print(K,scores.mean())

Vemos que el K optimo es el K=8. Ahora vamos a chequear nuestra elección.

In [ ]:
K_optimo = cv_k_scores.index(max(cv_k_scores)) + 1
print("el K óptimo es K =", K_optimo)

### Objetivo
Para el valor de K obtenido, ¿cuál es el porcentaje de aciertos en el conjunto de testeo

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
# Entrenamos el modelo con el valor óptimo de K
neighbor = KNeighborsClassifier(n_neighbors=K_optimo)
neighbor.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = neighbor.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
print("Porcentaje de acierto en el conjunto de testeo: {:.2f}%".format(accuracy * 100))

### Recomendaciones de jugadores

#### 8) Trabajamos ahora con el dataset transfermarkt_fbref_201920.csv que incluye la valuación de los jugadores. Una de las aplicaciones más comunes de análisis de datos en el fútbol es para obtener recomendaciones de jugadores a comprar.

In [ ]:
data = pd.read_csv("transfermarkt_fbref_201920.csv", delimiter=";")
data.head()

### Objetivo
En 2021 Messi fue transferido del Barcelona al PSG. Basándose en los datos disponibles, recomendarle a Barcelona un jugador de características similares a Messi

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


Para encontrar jugadores con características parecidas a Messi vamos a utilizar PCA teniendo en cuenta solo las variables numericas. Luego de esos jugadores vamos a ver cual tiene un valor de mercado menor al de Messi y juega en una posición similar.

In [ ]:
data2 = data.set_index("player")

In [ ]:
# Sacamos las columnas con valores no numericos
X = data2.drop(columns=["nationality","position","squad","position2", "foot", "league","Season"])

# Le sacamos las comas a los numeros dado que sino nos provocaba un error al correr el codigo
X = X.map(lambda x: float(str(x).replace(',', '')) if isinstance(x, str) else x)
X = X.dropna(axis=1)

# Aplicamos PCA ajustando a un 90%
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X)

# Creamos el nuevo dataframe
data2_pca = pd.DataFrame(X_pca, index=data2.index)

In [ ]:
jugadores_elegidos = []

# Definimos los datos de Messi y el rango que le daremos al "reemplazo"
stat_messi = data2_pca.loc["Lionel Messi", 0]
limite_inf = stat_messi * 0.75
limite_sup = stat_messi * 1.25

# Iteramos sobre cada jugador
for jugador in data2_pca.index:
        
    if jugador == "Lionel Messi":
        continue
    
    stats_cercanas_a_messi = 0

    stat_jugador = data2_pca.loc[jugador, 0]
    
    
    # Queremos solo los valores escalares
    if not isinstance(stat_jugador, (int, float)):
       continue
    
    # Verificamos el nuevo jugador entre en nuestros límites
    if limite_inf <= stat_jugador <= limite_sup:
        jugadores_elegidos.append(jugador)
        
        
print(jugadores_elegidos)

Podemos ver que los jugadores que entran en nuestro rango son: Oblak, Griezmann, Sancho, Alexander-Arnold, Mané, Salah, De Bruyne, Sterling, Neymar y Kane. De estos, vamos a descartar a Oblak y Alexander-Arnold ya que son arquero y defensor respectivamente. Vamos a ver el valor de mercado de los demás y luego vamos a dar el elegido para ser el reemplazo de Lionel Messi.

In [ ]:
valor_messi = data2.loc["Lionel Messi", "value"]

jugadores = []

for jugador in ["Antoine Griezmann","Jadon Sancho","Sadio Mané","Mohamed Salah","Kevin De Bruyne","Harry Kane","Raheem Sterling","Neymar"]:
    valor_jugador = data2.loc[jugador, "value"]
    if valor_jugador < valor_messi:
        jugadores.append(jugador)

print(jugadores)

Luego de finalizar el análisis, podemos decir que el reemplazo de Messi debe ser Antoine Griezmann, ya que es el único jugador que cumplió con todos los requisitos solicitados, y además juega en la misma posición que Lionel.

### Objetivo
Queremos elaborar un modelo para detectar jugadores “baratos”, es decir cuya valuación en el mercado (columna value) sea inferior a que la que nosotros estimemo

### Enfoque
Se implementa el procedimiento descrito, documentando las decisiones de preprocesamiento y validando los resultados con visualizaciones y métricas cuando corresponde.


In [ ]:
data2.head()

Intentaremos hacer redes neuronales...

In [ ]:
label = LabelEncoder()

columnas = ["nationality","position","squad","position2", "foot", "league"]

for col in columnas:
    data2[col + "_nueva"] = label.fit_transform(data2[col])
    data2.drop(col, axis=1, inplace=True)
    
data2.head()

In [ ]:
X = data2.drop(["value","Season"],axis=1)
y = data2["value"]

# Le saco las comas
X = X.map(lambda x: float(str(x).replace(',', '')) if isinstance(x, str) else x)

X_train, X_test, y_train, y_test = train_test_split_scale_center(X, y, test_size=0.2, random_state=42)

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

modelo = model.compile(optimizer='adam', loss='mean_squared_error')

hist = model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=50, batch_size=32, validation_split=0.2, verbose=0)

In [ ]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"El error cuadratico medio es de: {rmse:.2f}")

Vemos que no quedó muy bien asique probamos con regresión Ridge.

In [ ]:
def predecir(jugador):
    X = data2.drop(["value","Season"],axis=1)
    y = data2["value"]

    X = X.map(lambda x: float(str(x).replace(',', '')) if isinstance(x, str) else x)
    X = X.dropna(axis=1)

    alphas = [1,10,100,500,1000,2000,5000,7000,10000,20000,50000]
    cv = KFold(n_splits=5, random_state=42, shuffle=True)

    error_cuadratico_medio = float('inf')

    for alpha in alphas:
        modelo = linear_model.Ridge(alpha = alpha, fit_intercept = False)    
        rmse = np.zeros(cv.get_n_splits())

        ind = 0
    
        for train_index, val_index in cv.split(X):
            X_train, X_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
            modelo.fit(X_train, y_train)
    
            y_pred_val = modelo.predict(X_val)
            rmse[ind] = np.sqrt(mean_squared_error(y_val, y_pred_val))
            ind += 1
    
        if rmse.mean() < error_cuadratico_medio:
            error_cuadratico_medio = rmse.mean()
            alpha_optimo = alpha

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    modelo = linear_model.Ridge(alpha = alpha_optimo, fit_intercept = False)
    modelo.fit(X_train, y_train)
    y_pred_test = modelo.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    valor = data2.loc[jugador,"value"]
    
    return print(f"El valor de {jugador} en el mercado es de {valor}$ y nuestra predicción fue de {valor + rmse:.0f}$")

print(predecir("Lionel Messi"))

## Resultados principales (resumen)

- **Clustering**: se identifican perfiles de jugadores con patrones diferenciados en el espacio de características.
- **Clasificación**: se entrena un modelo para predecir posición/rol utilizando validación cruzada y métricas estándar.
- **Recomendación**: se propone un ranking de jugadores similares incorporando criterios de rendimiento y costo.
- **Valuación**: se estima valor de mercado aproximado a partir de variables predictoras relevantes.

> Completar con métricas/números concretos (accuracy/RMSE/ejemplos) una vez fijada la configuración final.
